# Audio Augmentation

## Module Used
https://pythonrepo.com/repo/iver56-audiomentations-python-audio

In [1]:
import sys
import os
from audiomentations import Compose, AddGaussianNoise, Trim, ClippingDistortion, Normalize, Gain, AddBackgroundNoise
import librosa as lb
import soundfile as sf
import glob

sys.path.append(os.path.abspath(os.path.join('..')))
from scripts.logger_creator import CreateLogger

C:\Python38\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
# Initializing Logger
logger = CreateLogger('AudioAugmentor', handlers=1)
logger = logger.get_default_logger()


In [6]:
class AudioAugmentor():
    def __init__(self, audios_path:str='../data/train/wav/', save_path:str='../data/train/clean_wav/', sample_rate:int=16000) -> None:
        try:
            self.SAMPLE_RATE = sample_rate
            self.audios_path = glob.glob(audios_path + '*.wav')
            self.save_path = save_path

            logger.info('Successfully Created AudioAugmentor Class Object')

        except Exception as e:
            logger.exception('Failed to Create AudioAugmentor Class Object')

    def generate_composes(self):
        # Augmentations
        # Clean Audio
        try:
            self.clean_audio = Compose([
                Trim(),
                ClippingDistortion(min_percentile_threshold=10, max_percentile_threshold=10, p=0.5),
                Normalize()
            ])

            # Adding Gaussian Noise
            self.guassian_noise = Compose([
                Trim(),
                AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
                Normalize()
            ])

            # Increase Volume (Gain)
            self.increase_volume = Compose([
                Trim(),
                Gain(min_gain_in_db=-10, max_gain_in_db=14),
                ClippingDistortion(min_percentile_threshold=10, max_percentile_threshold=10, p=0.5),
                Normalize()
            ])

            # Lower Volume (Gain)
            self.decrease_volume = Compose([
                Trim(),
                Gain(min_gain_in_db=-20, max_gain_in_db=4),
                ClippingDistortion(min_percentile_threshold=10, max_percentile_threshold=10, p=0.5),
                Normalize()
            ])

            # Add Background Noise
            self.background_noise_1 = Compose([
                Trim(),
                AddBackgroundNoise(sounds_path='../data/background_noises/bg1/', min_snr_in_db=3, max_snr_in_db=30,
                                p=0.5, lru_cache_size=2,),
                Normalize()
            ])

            self.background_noise_2 = Compose([
                Trim(),
                AddBackgroundNoise(sounds_path='../data/background_noises/bg2/', min_snr_in_db=3, max_snr_in_db=30,
                                p=0.5, lru_cache_size=2,),
                Normalize()
            ])

            self.background_noise_3 = Compose([
                Trim(),
                AddBackgroundNoise(sounds_path='../data/background_noises/bg3/', min_snr_in_db=3, max_snr_in_db=30,
                                p=0.5, lru_cache_size=2,),
                Normalize()
            ])

            logger.info('Created 7 different Types of Audio Compositions Successfully')
        
        except Exception as e:
            logger.exception('Failed to create audio compositions')

    def augment_and_save(self):
        try:
            for audio_path in self.audios_path:
                audio, sr = lb.load(audio_path, sr=self.SAMPLE_RATE, mono=False)
                name = audio_path[:-4].split('wav')[1][1:]

                # Clean Audio
                sf.write(self.save_path + name + '#clean.wav', self.clean_audio(samples=audio,
                        sample_rate=self.SAMPLE_RATE), self.SAMPLE_RATE, subtype='PCM_24')
                # Gaussain Noise
                sf.write(self.save_path + name + '#gau.wav',
                        self.guassian_noise(samples=audio, sample_rate=self.SAMPLE_RATE), self.SAMPLE_RATE, subtype='PCM_24')
                # Increased Volume
                sf.write(self.save_path + name + '#vinc.wav',
                        self.increase_volume(samples=audio, sample_rate=self.SAMPLE_RATE), self.SAMPLE_RATE, subtype='PCM_24')
                # Decreased Volume
                sf.write(self.save_path + name + '#vdec.wav',
                        self.decrease_volume(samples=audio, sample_rate=self.SAMPLE_RATE), self.SAMPLE_RATE, subtype='PCM_24')
                # Background Noise 1
                sf.write(self.save_path + name + '#bg1.wav',
                        self.background_noise_1(samples=audio, sample_rate=self.SAMPLE_RATE), self.SAMPLE_RATE, subtype='PCM_24')
                # Background Noise 2
                sf.write(self.save_path + name + '#bg2.wav',
                        self.background_noise_2(samples=audio, sample_rate=self.SAMPLE_RATE), self.SAMPLE_RATE, subtype='PCM_24')
                # Background Noise 3
                sf.write(self.save_path + name + '#bg3.wav',
                        self.background_noise_3(samples=audio, sample_rate=self.SAMPLE_RATE), self.SAMPLE_RATE, subtype='PCM_24')

            logger.info('Successfully Augemented and Saved all Audio Files')

        except Exception as e:
            logger.exception(f'Failed to Augment Audio file: {audio_path}')

    def generate_augmentations(self):
        self.generate_composes()
        self.augment_and_save()


In [7]:
audio_aug = AudioAugmentor()

AudioAugmentor:INFO->Successfully Created AudioAugmentor Class Object


In [8]:
audio_aug.generate_augmentations()

AudioAugmentor:INFO->Created 7 different Types of Audio Compositions Successfully
AudioAugmentor:INFO->Successfully Augemented and Saved all Audio Files
